In [134]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os

def clean_host(host):
    return host.split('.')[-2]

def get_site(site_url):
    return site_url.split(".")[1]

def is_third_party(origin, cookie_host):
    if origin in cookie_host:
        return False
    return True


In [135]:
sql_file = 'exp-data/nyt-t1.sqlite' 
    
###### javascript cookies  ######
query = """SELECT sv.site_url, sv.visit_id,
        jsc.host, jsc.name, jsc.value
        FROM javascript_cookies as jsc LEFT JOIN site_visits as sv
        ON sv.visit_id = jsc.visit_id
        """

conn = sqlite3.connect(sql_file)

cookies = pd.read_sql_query(query, conn)
cookies

,site_url,visit_id,host,name,value
0,https://www.webmd.com/search/search_results/de...,1,.www.webmd.com,__cfduid,de1f86f9e699cddfc6fcaeed623176bae1606506089
1,https://www.webmd.com/search/search_results/de...,1,.js.webmd.com,__cfduid,d8d4ebd1e437ddcb310e31be0d82e1cdb1606506089
2,https://www.webmd.com/search/search_results/de...,1,.img.webmd.com,__cfduid,d5b4e59d78d86d6c5ab29b6c3e948cd431606506089
3,https://www.webmd.com/search/search_results/de...,1,.css.webmd.com,__cfduid,d74794a4d59fa87e1456c8b173f3ff20e1606506089
4,https://www.webmd.com/search/search_results/de...,1,.webmd.com,test,cookie
...,...,...,...,...,...
2148,https://www.nytimes.com/,3,.bluekai.com,bku,5LD99nOUyPI+cnRG
2149,https://www.nytimes.com/,3,.bluekai.com,bku,5LD99nOUyPI+cnRG
2150,https://www.nytimes.com/,3,.pubmatic.com,pi,156011:2
2151,https://www.nytimes.com/,3,.ads.pubmatic.com,PMFREQ_ON,YES


In [136]:
cookies['origin_site'] = list(map(lambda x: get_site(x), cookies.site_url))
cookies["host"] = list(map(lambda x: clean_host(x), cookies.host.tolist()))
cookies["is_third_party"] = list(map(lambda a, b: is_third_party(a, b), cookies.origin_site, cookies.host))

In [137]:
# 95 unique host -> after clean the host: 67
# cookies.host.unique()

In [138]:
cookies = cookies[["visit_id", "origin_site", "host", "name", "value", "is_third_party"]]

In [139]:
# drop duplicate: 1396 -> 539
cookies = cookies.drop_duplicates().reset_index(drop = True)

In [140]:
cookies

,visit_id,origin_site,host,name,value,is_third_party
0,1,webmd,webmd,__cfduid,de1f86f9e699cddfc6fcaeed623176bae1606506089,False
1,1,webmd,webmd,__cfduid,d8d4ebd1e437ddcb310e31be0d82e1cdb1606506089,False
2,1,webmd,webmd,__cfduid,d5b4e59d78d86d6c5ab29b6c3e948cd431606506089,False
3,1,webmd,webmd,__cfduid,d74794a4d59fa87e1456c8b173f3ff20e1606506089,False
4,1,webmd,webmd,test,cookie,False
...,...,...,...,...,...,...
835,3,nytimes,pubmatic,SPugT,1606506426,True
836,3,nytimes,iasds01,AC,1,True
837,3,nytimes,iasds01,DMADT,n+rOZk98RB69wOmiXlusmA,True
838,3,nytimes,iasds01,UT,"""""",True


In [141]:
# cookies[cookies.is_third_party]

In [142]:
# make sure to use brackets when filtering visit_id == 3
temp = cookies[(cookies.visit_id == 3) & cookies.is_third_party]
temp.groupby('host',as_index=False)["host", "name", "value"].size().nlargest(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


host
pubmatic          18
rubiconproject    14
media              8
adsrvr             7
casalemedia        5
dtype: int64

In [131]:
cookies.groupby('origin_site')["host", "name", "value"].nunique()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,host,name,value
origin_site,,,
mayoclinic,11,47,76
nytimes,25,80,94
webmd,57,186,322


In [20]:
# cookies.groupby('origin_site').size()

In [16]:
# big_table = []

# data_path = "exp-data/"

# def extract_unique_cookie_pairs(type_name, exp_group, exp_num):
#     file_name = type_name + '-' + exp_group + str(exp_num)
#     print(file_name)
#     conn = sqlite3.connect(data_path + file_name +".sqlite")
#     javascript_cookies = pd.read_sql_query(
#     "select visit_id, host, name, value from javascript_cookies;", conn)
#     javascript_cookies = javascript_cookies.drop_duplicates()
#     df = javascript_cookies.groupby('visit_id')["host", "name", "value"].nunique()
#     pairs = javascript_cookies.groupby('visit_id').size().values
#     df["pairs"] = pairs
#     df['origin_site'] = ['webmd', 'mayoclinic', type_name]
#     df['exp_type'] = [type_name] * 3
#     df['exp_group'] = [exp_group] * 3
#     df['exp_index'] = [exp_num] * 3
#     return df

In [19]:
# exp_types = ['nyt', 'forbes', 'washington']
# exp_groups = ['t', 'c']
# for i in range(1, 7):
#     for t in exp_types:
#         for g in exp_groups:
#             big_table.append(extract_unique_cookie_pairs(t, g, i))

In [18]:
# result = pd.concat(big_table)

In [21]:
# result.to_csv("cookie_result-6.csv", index=False)

In [20]:
# result

,host,name,value,pairs,origin_site,exp_type,exp_group,exp_index
visit_id,,,,,,,,
1,78,190,355,394,webmd,nyt,t,1
2,85,168,278,312,mayoclinic,nyt,t,1
3,43,118,125,139,nyt,nyt,t,1
1,74,186,322,351,webmd,nyt,c,1
2,17,47,76,88,mayoclinic,nyt,c,1
3,33,80,94,100,nyt,nyt,c,1
1,80,204,351,389,webmd,forbes,t,1
2,76,153,237,265,mayoclinic,forbes,t,1
3,82,225,251,305,forbes,forbes,t,1
